In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import movie_reviews
# text
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer, TfidfVectorizer
# classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC
# classterization
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering, DBSCAN, MiniBatchKMeans
# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, label_binarize, OneHotEncoder
# mertrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# embedding vectors
# Важно
# It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data)  
# to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high.  
# This will suppress some noise and speed up the computation of pairwise distances between samples
from sklearn.manifold import TSNE
# reduce the dimensionality
from sklearn.decomposition import PCA, TruncatedSVD

import string
import random
from sklearn import datasets
import pandas as pd
import numpy as np
from datetime import date,datetime
import matplotlib.pyplot as plt
import seaborn as sns

from pylab import rcParams
rcParams['figure.figsize'] = 8, 6

In [2]:
# Загружаем набор данных
iris_df = datasets.load_iris()

### Lable encoding

если порядок совпадает с ожидаемым, то есть по алфавиту буквы идут и подходит, что будет от 0 до n  
или нам не важно какая буква будет с какой цифорой

In [4]:
le = LabelEncoder()

l = ['a', 'c', 'b', 'd', 'e']

le.fit_transform(l)

array([0, 2, 1, 3, 4], dtype=int64)

если нам нужно правильный порядок сделать, то используем label_binarize  
label_binarize делает one hot encoding, и если нам нужно от 0 до n, то используем argmax

In [11]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad']).argmax(axis=1)

array([3, 0, 2, 1, 4], dtype=int64)

In [12]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad'])

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1]])

## Сокращение размерности

PCA

In [16]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
pca.fit(X)
PCA(n_components=2)
# Чтобы проверить соотношение дисперсии, объясненную этим СПС, мы можем рассмотреть pca.explained_variance_ratio_ :
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print()
# линейные комбинации
# Обратите внимание, что первые два компонента в каждом векторе на несколько порядков больше других, что показывает,   
# что СПС признал, что дисперсия содержится главным образом в первых двух столбцах
print(pca.components_)

[0.99244289 0.00755711]
[6.30061232 0.54980396]

[[-0.83849224 -0.54491354]
 [ 0.54491354 -0.83849224]]


In [17]:
# Загрузка данных
data = datasets.load_iris()
X, y = data.data, data.target

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Обучение SVM (C-Support Vector Classification) на данных после PCA
svm = SVC()
svm.fit(X_train_pca, y_train)

# Оценка производительности модели
accuracy = svm.score(X_test_pca, y_test)
print(f'Accuracy after PCA: {accuracy:.2f}')

Accuracy after PCA: 1.00


TruncatedSVD

In [14]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
np.random.seed(0)
X_dense = np.random.rand(100, 100)
X_dense[:, 2 * np.arange(50)] = 0
X = csr_matrix(X_dense)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(X)
TruncatedSVD(n_components=5, n_iter=7, random_state=42)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

[0.01570766 0.05122679 0.04998062 0.04795064 0.04539933]
0.21026503465070323
[35.24105443  4.5981613   4.54200434  4.44866153  4.32887456]


## Классификация

In [ ]:
# Пример данных
nltk.download('movie_reviews')

lemma = nltk.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)

# Загрузка данных
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
# Подготовка данных
texts = [' '.join(doc) for doc, _ in documents]
labels = [1 if category == 'pos' else 0 for _, category in documents]
prep_text = []
# Токенизация и лематизация
for sentence in texts:
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    lem_tokens = [lemma.lemmatize(token) for token in tokens]
    lem_filt_tokens = [token for token in lem_tokens if token not in stop_words and token not in punctuation]
    prep_sentence = ' '.join(lem_filt_tokens)
    prep_text.append(prep_sentence)
   
# Создание BoW модели
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(prep_text)

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(bow, labels, test_size=0.3)


MultinomialNB

In [ ]:
# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

Accuracy: 0.8033333333333333
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       304
           1       0.81      0.79      0.80       296

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600



LogisticRegression

In [ ]:
# Обучение классификатора
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

Accuracy: 0.835
              precision    recall  f1-score   support

           0       0.83      0.85      0.84       304
           1       0.84      0.82      0.83       296

    accuracy                           0.83       600
   macro avg       0.84      0.83      0.83       600
weighted avg       0.84      0.83      0.83       600



LinearDiscriminantAnalysis

In [ ]:
# Обучение классификатора
classifier = LinearDiscriminantAnalysis()
classifier.fit(X_train.toarray(), y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

KNeighborsClassifier

In [ ]:
# Обучение классификатора
classifier = KNeighborsClassifier()
classifier.fit(X_train.toarray(), y_train)

# Оценка классификатора
predictions = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

print(classification_report(y_test, predictions, ))

## Класстеризация

По ссылке можно посмотреть базовые преимущества и недостатки методов кластеризации <br>
https://scikit-learn.org/stable/modules/clustering.html

KMeans

In [9]:
iris_df.data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
# Описываем модель
model = KMeans(n_clusters=3)
# Проводим моделирование
model.fit(iris_df.data)
# Предсказание на всем наборе данных
all_predictions = model.predict(iris_df.data)
all_predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1])

### Пример класстеризации